<a href="https://colab.research.google.com/github/innovate-data/PDM/blob/main/Streamlit_Predictive_Maintenance_UI_Prototype_(Compressors).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import streamlit as st
import pandas as pd
import numpy as np # Required for placeholder data generation
from datetime import datetime, timedelta # Required for date calculations

# --- Configuration ---
st.set_page_config(page_title="Compressor PdM Dashboard", layout="wide")

# --- Helper Functions (Example: Determine status color) ---
def get_status_color(status):
    """Returns a color based on equipment status."""
    if status == "Critical":
        return "red"
    elif status == "Warning":
        return "orange"
    elif status == "Anomaly":
        return "purple" # Added status for anomalies
    else: # Normal
        return "green"

# --- Placeholder Data Generation ---
# In a real application, you would load this data from your backend/database
# where model predictions and anomaly flags are stored.
def load_data():
    """Generates sample data for the dashboard, specific to compressors."""
    num_compressors = 15
    equipment_ids = [f'COMP-{i:03d}' for i in range(1, num_compressors + 1)]
    statuses = np.random.choice(['Normal', 'Warning', 'Critical', 'Anomaly'], num_compressors, p=[0.6, 0.15, 0.1, 0.15])
    ruls = np.random.randint(5, 300, num_compressors)
    probs = np.round(np.random.uniform(1, 95, num_compressors), 1)
    pressures = np.round(np.random.uniform(80, 150, num_compressors), 1) # Example sensor
    temps = np.round(np.random.uniform(50, 90, num_compressors), 1)     # Example sensor
    vibrations = np.round(np.random.uniform(0.1, 5.0, num_compressors), 2) # Example sensor

    data = {
        'Equipment ID': equipment_ids,
        'Status': statuses,
        'Predicted RUL (Days)': ruls,
        'Failure Probability (%)': probs,
        'Pressure (PSI)': pressures,
        'Temperature (°C)': temps,
        'Vibration (mm/s)': vibrations,
        'Last Update': pd.to_datetime('today').strftime('%Y-%m-%d %H:%M')
    }
    # Ensure Critical/Anomaly status has low RUL and high probability for realism
    df = pd.DataFrame(data)
    critical_anomaly_mask = (df['Status'] == 'Critical') | (df['Status'] == 'Anomaly')
    warning_mask = df['Status'] == 'Warning'

    df.loc[critical_anomaly_mask, 'Predicted RUL (Days)'] = np.random.randint(1, 15, critical_anomaly_mask.sum())
    df.loc[critical_anomaly_mask, 'Failure Probability (%)'] = np.round(np.random.uniform(80, 99, critical_anomaly_mask.sum()), 1)
    df.loc[warning_mask, 'Predicted RUL (Days)'] = np.random.randint(15, 60, warning_mask.sum())
    df.loc[warning_mask, 'Failure Probability (%)'] = np.round(np.random.uniform(50, 80, warning_mask.sum()), 1)

    # Calculate Predicted Maintenance Date
    today_date = datetime.now().date()
    df['Predicted Maintenance Date'] = df['Predicted RUL (Days)'].apply(lambda x: today_date + timedelta(days=int(x)))
    # Format date for display
    df['Predicted Maintenance Date'] = pd.to_datetime(df['Predicted Maintenance Date']).dt.strftime('%Y-%m-%d')


    return df

# --- Load Data ---
df_equipment = load_data()

# --- Sidebar for Configuration Inputs ---
st.sidebar.header("⚙️ Configuration")
st.sidebar.markdown("Set thresholds and parameters for the dashboard.")

# Example Configuration Inputs
prob_threshold = st.sidebar.slider("Failure Probability Alert Threshold (%)", 0, 100, 80)
rul_threshold = st.sidebar.number_input("RUL Alert Threshold (Days)", min_value=1, max_value=365, value=14)
selected_model = st.sidebar.selectbox("Select Predictive Model", ["Model A (Default)", "Model B (Experimental)"])
refresh_interval = st.sidebar.select_slider("Data Refresh Interval (minutes)", options=[5, 15, 30, 60], value=15)

st.sidebar.info(f"""
**Current Settings:**
- Probability Threshold: {prob_threshold}%
- RUL Threshold: {rul_threshold} days
- Model: {selected_model}
- Refresh: Every {refresh_interval} mins
""")

# --- Dashboard UI ---

# Title
st.title("Industrial Compressor Predictive Maintenance")
st.markdown(f"*Last Updated: {df_equipment['Last Update'].iloc[0]}*")

st.divider()

# --- Alerts Section (Based on Config) ---
st.subheader("🚨 Priority Alerts")
# Filter based on configuration thresholds OR Critical/Anomaly status
alert_equipment = df_equipment[
    (df_equipment['Failure Probability (%)'] >= prob_threshold) |
    (df_equipment['Predicted RUL (Days)'] <= rul_threshold) |
    (df_equipment['Status'].isin(['Critical', 'Anomaly']))
].sort_values(by='Predicted RUL (Days)') # Sort by urgency

if not alert_equipment.empty:
    st.warning(f"Found {len(alert_equipment)} compressors needing attention based on thresholds (Prob >= {prob_threshold}% or RUL <= {rul_threshold} days) or Critical/Anomaly status.")
    for index, row in alert_equipment.iterrows():
        alert_color = get_status_color(row['Status'])
        st.markdown(
            f"<p style='color:{alert_color}; border:1px solid {alert_color}; padding: 5px; border-radius: 5px;'>"
            f"<strong>{row['Equipment ID']}</strong>: Status <strong>{row['Status']}</strong>. "
            f"Predicted RUL: {row['Predicted RUL (Days)']} days. "
            f"Failure Probability: {row['Failure Probability (%)']}%. "
            f"Predicted Maint. Date: {row['Predicted Maintenance Date']}"
            f"</p>",
            unsafe_allow_html=True
        )

else:
    st.success("No equipment meets the current alert criteria.")

st.divider()

# --- Anomaly Detection Section ---
st.subheader("🔬 Detected Anomalies")
anomaly_equipment = df_equipment[df_equipment['Status'] == 'Anomaly']

if not anomaly_equipment.empty:
    st.info(f"Detected anomalies in {len(anomaly_equipment)} compressors. These may require investigation even if RUL/Probability are not yet critical.")
    st.dataframe(anomaly_equipment[['Equipment ID', 'Pressure (PSI)', 'Temperature (°C)', 'Vibration (mm/s)', 'Last Update']])
else:
    st.success("No specific anomalies detected currently.")


st.divider()

# --- Overview Section ---
st.subheader("🏭 Compressor Fleet Overview")

# More customized display using columns
col1, col2, col3 = st.columns(3)
total_monitored = len(df_equipment)
warning_count = len(df_equipment[df_equipment['Status'] == 'Warning'])
critical_count = len(df_equipment[df_equipment['Status'] == 'Critical']) # Excludes Anomaly from main count

col1.metric("Total Compressors Monitored", total_monitored)
col2.metric("Compressors with Warning", warning_count)
col3.metric("Compressors with Critical Status", critical_count, delta=critical_count, delta_color="inverse" if critical_count > 0 else "off")


# Display styled table (optional: using HTML for colors)
st.markdown("#### Compressor Status Details")

# Convert DataFrame to HTML with colored status (requires unsafe_allow_html=True)
def dataframe_to_html_with_status_and_date(df):
    html = "<table><thead><tr><th>Equipment ID</th><th>Status</th><th>Predicted RUL (Days)</th><th>Failure Probability (%)</th><th>Predicted Maintenance Date</th></tr></thead><tbody>"
    for _, row in df.iterrows():
        color = get_status_color(row['Status'])
        html += (f"<tr><td>{row['Equipment ID']}</td>"
                 f"<td style='color:{color}; font-weight:bold;'>{row['Status']}</td>"
                 f"<td>{row['Predicted RUL (Days)']}</td>"
                 f"<td>{row['Failure Probability (%)']}</td>"
                 f"<td>{row['Predicted Maintenance Date']}</td></tr>")
    html += "</tbody></table>"
    return html

# Display the HTML table
st.markdown(dataframe_to_html_with_status_and_date(df_equipment), unsafe_allow_html=True)


st.divider()

# --- Detailed View Section (Placeholder) ---
st.subheader("⚙️ Detailed Compressor View")

# Dropdown to select equipment
selected_equipment_id = st.selectbox("Select Compressor ID for Details:", df_equipment['Equipment ID'])

if selected_equipment_id:
    # Get data for the selected equipment
    equipment_details = df_equipment[df_equipment['Equipment ID'] == selected_equipment_id].iloc[0]

    st.write(f"**Showing details for:** {equipment_details['Equipment ID']}")
    st.write(f"**Current Status:** {equipment_details['Status']}")
    st.write(f"**Predicted RUL:** {equipment_details['Predicted RUL (Days)']} Days")
    st.write(f"**Failure Probability:** {equipment_details['Failure Probability (%)']}%")
    st.write(f"**Predicted Maintenance Date:** {equipment_details['Predicted Maintenance Date']}")

    # Placeholder for Charts (e.g., Sensor Trends, Prediction History)
    st.markdown("**Sensor Readings & Trends (Placeholder):**")
    st.write("*(Imagine charts showing Pressure, Temperature, Vibration trends, RUL trend, etc. here)*")
    # Example: Placeholder for a line chart using compressor-specific data
    # In a real app, you'd fetch historical data for the selected compressor
    chart_data = pd.DataFrame({
        'Pressure (PSI)': np.random.normal(equipment_details['Pressure (PSI)'], 5, 20),
        'Temperature (°C)': np.random.normal(equipment_details['Temperature (°C)'], 2, 20),
        'Vibration (mm/s)': np.random.normal(equipment_details['Vibration (mm/s)'], 0.5, 20)
        })
    st.line_chart(chart_data)

    # Placeholder for Contributing Factors
    st.markdown("**Contributing Factors (Placeholder):**")
    st.write("*(Imagine a list or table showing factors influencing the prediction here)*")


# --- Footer ---
st.caption("Prototype UI generated by AI")

ModuleNotFoundError: No module named 'streamlit'